In [85]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')

In [86]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.prompts import load_prompt
# from langchain_text_splitters.base import TextSplitter
# from pymongo import MongoClient
# from pymongo.mongo_client import MongoClient
# from pymongo.server_api import ServerApi
import json
import time
import datetime
import pypdf

In [87]:
prompt_template = "templates/template.json"
faiss_index = "faiss_index"
data_source = "data\swapnil profile.csv"
pdf_source =  "data\Swapnil Banduke Resume copy.pdf"


In [88]:
pdf_source

'data\\Swapnil Banduke Resume copy.pdf'

In [89]:
def is_valid_json(data):
    try:
        json.loads(data)
        return True
    except json.JSONDecodeError:
        return False

In [90]:
import os
os.environ['OPENAI_API_KEY'] = "key"

In [91]:
embeddings = OpenAIEmbeddings()

In [92]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=400,
    chunk_overlap=40
)

In [93]:
pdf_loader = PyPDFLoader(pdf_source)
pdf_data = pdf_loader.load_and_split(text_splitter=text_splitter)

Created a chunk of size 499, which is longer than the specified 400


In [94]:
pdf_data

[Document(page_content='SwapnilC.Banduke\nTexas,USA|Linkedin|Github|Portfolio||+1(469)-427-1328swapnil.banduke@utdallas.edu\nEDUCATIONUniversityofTexasatDallas,MasterofScience,BusinessAnalyticsMay2025Coursework:Predictive&PrescriptiveAnalysis,BAwithR,AdvancedStatistics,PythonGPA3.9/4.0GovernmentCollegeofEngineering,Karad,IndiaJun2021BachelorofTechnology,ElectricalEngineeringGPA3.6/4.0', metadata={'source': 'data\\Swapnil Banduke Resume copy.pdf', 'page': 0}),
 Document(page_content='TECHNICALSKILLSProgrammingLanguages:SQL(DataAnalysis,QueryOptimization),Python(Pandas,NumPy,SciPy,MatplotlibforDataManipulationandVisualization),R(StatisticalComputing)Databases:MSSQLServer,MySQL,Oracle,PostgreSQLDataAnalytics&VisualizationTools:Tableau(InteractiveDashboards,DataInsights),PowerBI(DataReporting,Visualization),RStudio,AdvancedMSExcel(Macros,PivotTables,VLOOKUP,ConditionalFormatting,VBAAutomation),MSOffice(DataAggregation,SpreadsheetModeling),Snowflake(Datasharing,Datawarehousing)', metadata={

In [95]:
csv_loader = CSVLoader(file_path=data_source, encoding="utf-8")
csv_data = csv_loader.load()

In [96]:
csv_data

[Document(page_content='Section: Personal Details\nDetail: Name\nDescription: Swapnil C. Banduke', metadata={'source': 'data\\swapnil profile.csv', 'row': 0}),
 Document(page_content='Section: Personal Details\nDetail: Phone\nDescription: +1 (469)-427-1328', metadata={'source': 'data\\swapnil profile.csv', 'row': 1}),
 Document(page_content='Section: Personal Details\nDetail: Location\nDescription: Texas, USA', metadata={'source': 'data\\swapnil profile.csv', 'row': 2}),
 Document(page_content='Section: Personal Details\nDetail: Email\nDescription: swapnil.banduke@utdallas.edu', metadata={'source': 'data\\swapnil profile.csv', 'row': 3}),
 Document(page_content='Section: Personal Details\nDetail: LinkedIn\nDescription: Linkedin URL', metadata={'source': 'data\\swapnil profile.csv', 'row': 4}),
 Document(page_content='Section: Personal Details\nDetail: Github\nDescription: Github URL', metadata={'source': 'data\\swapnil profile.csv', 'row': 5}),
 Document(page_content='Section: Personal

In [97]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [98]:
prompt = load_prompt("templates/template.json")

In [99]:
# Combine PDF and CSV data
data = pdf_data + csv_data

# Create embeddings for the docs
vectors = FAISS.from_documents(data, embeddings)
vectors.save_local("faiss_index")

In [100]:
retriever = vectors.as_retriever(search_type="similarity", search_kwargs={"k":6, "include_metadata":True, "score_threshold":0.6})
chain = ConversationalRetrievalChain.from_llm(llm=ChatOpenAI(temperature=0.5,model_name='gpt-3.5-turbo-0125', openai_api_key=openai_api_key), 
                                                retriever=retriever,return_source_documents=True,verbose=True,chain_type="stuff",
                                                max_tokens_limit=4097, combine_docs_chain_kwargs={"prompt": prompt})


d:\MLops\Github projects\SwapnilGPT\SwapnilGPT\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [101]:
chat_history = []

result = chain({
    "system": "You are a Resume Bot, a comprehensive, interactive resource for providing info based on provided data only. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.",
    "question": "How swapnil is doing in Gen ai",
    "chat_history": chat_history
})

d:\MLops\Github projects\SwapnilGPT\SwapnilGPT\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Resume Bot, a comprehensive, interactive resource for exploring Swapnil C. Banduke's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge. 
 Human: Take a deep breath and do the following step by step these 4 steps:  
 1. Read the context below 
 2. Answer the question with detail using the provided Help Centre information 
 3. Make sure to nicely format the output as a three paragraph answer and try to incorporate the STAR format. 
 4. Provide 3 examples of questions users can ask about Swapnil Banduke based on the questions from context. Context: 
 ~~~ Section: How swapnil is doing in Gen ai
Detail: Swapnil developed a proprietary Generative AI tool utilizing a RAG model for text mining on weekly magazine reports, resulting in a 40% increase in efficiency for extra

In [102]:
result

{'system': 'You are a Resume Bot, a comprehensive, interactive resource for providing info based on provided data only. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.',
 'question': 'How swapnil is doing in Gen ai',
 'chat_history': [],
 'answer': '{\n  "answered": true,\n  "response": "Swapnil has shown great proficiency in Generative AI by developing a proprietary tool using a RAG model for text mining, resulting in a 40% increase in efficiency for extracting industry-relevant information. This project was completed during his time as an Applied Insight Intern at Eversana, showcasing his strong background in data analytics and AI.",\n  "questions": [\n    "What specific tools and technologies did Swapnil use to develop the Generative AI tool for text mining?",\n    "How did Swapnil\'s project with the Walmart Sales Assistance AI chatbot enhance customer service?",\n    "Can Swapnil provide more details about his e

In [103]:
result['answer']

'{\n  "answered": true,\n  "response": "Swapnil has shown great proficiency in Generative AI by developing a proprietary tool using a RAG model for text mining, resulting in a 40% increase in efficiency for extracting industry-relevant information. This project was completed during his time as an Applied Insight Intern at Eversana, showcasing his strong background in data analytics and AI.",\n  "questions": [\n    "What specific tools and technologies did Swapnil use to develop the Generative AI tool for text mining?",\n    "How did Swapnil\'s project with the Walmart Sales Assistance AI chatbot enhance customer service?",\n    "Can Swapnil provide more details about his experience as a Data Analyst at Kirloskar Brothers Limited in Pune, India?"\n  ]\n}'

In [104]:
result['source_documents']

[Document(page_content='Section: How swapnil is doing in Gen ai\nDetail: Swapnil developed a proprietary Generative AI tool utilizing a RAG model for text mining on weekly magazine reports, resulting in a 40% increase in efficiency for extracting industry-relevant information. This project was done during his time as a Applied Insight Intern at Eversana. From the provided information, Swapnil seems to have a strong background in data analytics and AI, showcasing his skills and experience in the field.\nDescription: ', metadata={'source': 'data\\swapnil profile.csv', 'row': 28}),
 Document(page_content='Section: Personal Details\nDetail: Name\nDescription: Swapnil C. Banduke', metadata={'source': 'data\\swapnil profile.csv', 'row': 0}),
 Document(page_content='SwapnilC.Banduke\nTexas,USA|Linkedin|Github|Portfolio||+1(469)-427-1328swapnil.banduke@utdallas.edu\nEDUCATIONUniversityofTexasatDallas,MasterofScience,BusinessAnalyticsMay2025Coursework:Predictive&PrescriptiveAnalysis,BAwithR,Adv

In [105]:
data = json.loads(result["answer"])

In [106]:
data.get("response")

'Swapnil has shown great proficiency in Generative AI by developing a proprietary tool using a RAG model for text mining, resulting in a 40% increase in efficiency for extracting industry-relevant information. This project was completed during his time as an Applied Insight Intern at Eversana, showcasing his strong background in data analytics and AI.'

In [107]:
chat_history.append((result["question"], result["answer"]))

In [108]:
chat_history

[('How swapnil is doing in Gen ai',
  '{\n  "answered": true,\n  "response": "Swapnil has shown great proficiency in Generative AI by developing a proprietary tool using a RAG model for text mining, resulting in a 40% increase in efficiency for extracting industry-relevant information. This project was completed during his time as an Applied Insight Intern at Eversana, showcasing his strong background in data analytics and AI.",\n  "questions": [\n    "What specific tools and technologies did Swapnil use to develop the Generative AI tool for text mining?",\n    "How did Swapnil\'s project with the Walmart Sales Assistance AI chatbot enhance customer service?",\n    "Can Swapnil provide more details about his experience as a Data Analyst at Kirloskar Brothers Limited in Pune, India?"\n  ]\n}')]

In [109]:
result = chain({"system": 
        "You are a Resume Bot, a comprehensive, interactive resource for providing info based on provided data only. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.", 
                        "question": "Should i play advertisment on the youtube?",
                        "chat_history":chat_history})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How swapnil is doing in Gen ai
Assistant: {
  "answered": true,
  "response": "Swapnil has shown great proficiency in Generative AI by developing a proprietary tool using a RAG model for text mining, resulting in a 40% increase in efficiency for extracting industry-relevant information. This project was completed during his time as an Applied Insight Intern at Eversana, showcasing his strong background in data analytics and AI.",
  "questions": [
    "What specific tools and technologies did Swapnil use to develop the Generative AI tool for text mining?",
    "How did Swapnil's project with the Walmart Sales Assistance AI chatbot enhance customer service?",
    "Can Swapnil provide more details about his experience as a Data Analyst at Kirloskar Brothers Limite

In [110]:
result["answer"]

'{\n    "answered": true,\n    "response": "As a Resume Bot, I am not able to provide real-time advice on whether you should play an advertisement on YouTube. However, if you have questions related to Swapnil Banduke\'s background, skills, and expertise, feel free to ask for more information.",\n    "questions": [\n        "What impact did Swapnil Banduke\'s Tableau dashboard have at Eversana?",\n        "Can Swapnil Banduke explain their contribution to Eversana\'s Next Best Experience project?",\n        "How did Swapnil Banduke leverage machine learning and data analytics in their previous roles?"\n    ]\n}'

In [111]:
chat_history.append((result["question"], result["answer"]))

In [112]:
result = chain({"system": 
        "You are a Resume Bot, a comprehensive, interactive resource for providing info based on provided data only. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.", 
                        "question": "What is the best way to promote the drugs?",
                        "chat_history":chat_history})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How swapnil is doing in Gen ai
Assistant: {
  "answered": true,
  "response": "Swapnil has shown great proficiency in Generative AI by developing a proprietary tool using a RAG model for text mining, resulting in a 40% increase in efficiency for extracting industry-relevant information. This project was completed during his time as an Applied Insight Intern at Eversana, showcasing his strong background in data analytics and AI.",
  "questions": [
    "What specific tools and technologies did Swapnil use to develop the Generative AI tool for text mining?",
    "How did Swapnil's project with the Walmart Sales Assistance AI chatbot enhance customer service?",
    "Can Swapnil provide more details about his experience as a Data Analyst at Kirloskar Brothers Limite


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Resume Bot, a comprehensive, interactive resource for exploring Swapnil C. Banduke's background, skills, and expertise. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge. 
 Human: Take a deep breath and do the following step by step these 4 steps:  
 1. Read the context below 
 2. Answer the question with detail using the provided Help Centre information 
 3. Make sure to nicely format the output as a three paragraph answer and try to incorporate the STAR format. 
 4. Provide 3 examples of questions users can ask about Swapnil Banduke based on the questions from context. Context: 
 ~~~ Section: Questions
Detail: 
Description: 

Section: Work Experience
Detail: What impact did your Tableau dashboard have at Eversana?
Description: Improved marketing campaign effectiveness by 25% th

In [113]:
result["answer"]

'{\n    "answered": true,\n    "response": "As a Resume Bot, I must clarify that I am not Swapnil Banduke. However, based on their experience at Eversana, one effective way to promote drugs could be to utilize Tableau dashboards for real-time visualization of promotional metrics. This approach can help in optimizing marketing strategies and improving campaign effectiveness. Additionally, implementing ETL pipelines to gather affinity data and integrating it into a data warehouse like Snowflake can also enhance drug promotion efforts by providing valuable insights.",\n    "questions": [\n        "Can you elaborate on your experience with Tableau dashboards and their impact on marketing campaigns?",\n        "How did your work with ETL pipelines contribute to improving promotional strategies?",\n        "What role did XGBoost models play in achieving audience reach and revenue generation at Eversana?"\n    ]\n}'